In [1]:
from newspaper import Article
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException


https://github.com/YaKsenia/scraper_al_jazeera_news/tree/master

In [12]:
# keywords = ["Israel", "Palestine", "Gaza", "Hamas"]
keywords = ['https://www.aljazeera.com/search/palestine',
            'https://www.aljazeera.com/search/israel',
            'https://www.aljazeera.com/search/gaza',
            'https://www.aljazeera.com/search/hamas',
            ]

In [13]:
def scrape_links(url):
    driver = webdriver.Chrome(executable_path="/Users/alexanderkumar/chromedriver")

    def show_more_button_exists():
        try:
            # Check if the button is present in the DOM
            button = driver.find_element(By.CLASS_NAME, 'show-more-button')
            return True
        except NoSuchElementException:
            return False
        
    #open the link
    driver.get(url)
    wait = WebDriverWait(driver, 5)
    # driver.maximize_window()

    while show_more_button_exists():
        try:
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'show-more-button')))
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()
            print("Clicked!")
        except ElementClickInterceptedException:
            # Handle click interception, wait for overlay to disappear
            print("Click intercepted, waiting for overlay to disappear")
            try:
                # Wait for the overlay to disappear
                wait.until(EC.invisibility_of_element_located((By.XPATH, 'xpath_of_overlay_element')))
            except TimeoutException:
                print("Timed out waiting for overlay to disappear")
        time.sleep(2)

    wait.until(EC.presence_of_element_located((By.XPATH, '//a[@class="u-clickable-card__link"]')))
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    links = []
    for link in soup.find_all('a', class_='u-clickable-card__link'):
        links.append(link.get('href'))

    driver.quit()
    
    return links

In [14]:
all_links = []
for url in keywords:
    links = scrape_links(url)
    all_links = all_links + links

/var/folders/9c/5hbdnj_57xv861zqcz43bs6w0000gn/T/ipykernel_56205/1095628272.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/alexanderkumar/chromedriver")


Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!


In [17]:
full_texts = []
csv_name = 'Aljaz_text_data.csv'
start = 333

for i in range(start, len(all_links)):
    url = all_links[i]
    print(i, ": ", url)
    article = Article(url)
    time.sleep(6)
    article.download()
    article.parse()
    text = article.text
    full_texts.append(text)
    df_row = {"url" : [url], 
                "full_text": [text]}
    if i == 0:
        df_curr = pd.DataFrame(df_row)
        df_curr.to_csv(csv_name)  
    else:
        df_curr = pd.read_csv(csv_name, index_col=0)
        df_curr.loc[len(df_curr.index)] = [url, text]
        # df_curr.concat(df_row, ignore_index=True)
        df_curr.to_csv(csv_name)  


333 :  https://www.aljazeera.com/news/2023/12/1/israel-hamas-war-list-of-key-events-day-56
334 :  https://www.aljazeera.com/news/2023/12/2/israeli-bombs-hundreds-of-targets-in-gaza-as-truce-efforts-continue
335 :  https://www.aljazeera.com/gallery/2023/11/30/hamas-and-israel-free-more-detainees-in-sixth-captive-exchange
336 :  https://www.aljazeera.com/news/2023/11/15/biden-wholly-responsible-for-israeli-operation-in-gaza-hospital-hamas
337 :  https://www.aljazeera.com/news/2023/10/7/we-are-at-war-reactions-to-palestinian-hamas-surprise-attack-in-israel
338 :  https://www.aljazeera.com/news/2023/11/29/israel-hamas-truce-has-it-helped-relieve-gazas-humanitarian-crisis
339 :  https://www.aljazeera.com/news/2023/11/30/israel-hamas-war-list-of-key-events-day-55
340 :  https://www.aljazeera.com/news/2023/10/7/hamas-says-it-has-enough-israeli-captives-to-free-all-palestinian-prisoners
341 :  https://www.aljazeera.com/news/2023/10/7/what-happened-in-israel-a-breakdown-of-how-the-hamas-attack-